In [5]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

In [8]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_61b236dbccd64690ae8713bf329b81ad = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='pqwFJ9HIac3KnyauPag2N__eLVznWAGE-LXzxibTKBZX',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_61b236dbccd64690ae8713bf329b81ad.get_object(Bucket='default-donotdelete-pr-brw51ls98ewcag',Key='Geodefect_edited.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,GAUGE,UNLD_GAUGE,SUPER_ELE,TWIST_A,TWIST_B,SURF79_L,SURF79_R,ALIGN79_L,ALIGN79_R,ALIGN_31_L,...,SURF_31_R,SURF_62_L,SURF_62_R,CURVE,S_IDX,CANT_L,CANT_R,WEB_L,WEB_R,DEFECTS
0,976,303,0,0,0,0,0,0,0,0,...,0,0,0,0,963,75,179,-0.615378,0.967985,NONE
1,1021,305,0,0,0,0,0,0,0,0,...,0,0,0,0,981,55,163,-0.846215,0.783823,NONE
2,1140,325,0,0,0,0,0,0,0,0,...,0,0,0,0,999,80,159,-0.580411,0.752885,NONE
3,1120,335,0,0,0,0,0,0,0,0,...,0,0,0,0,1017,15,-40,0.000000,0.000000,NONE
4,1119,354,0,0,0,0,0,0,0,0,...,0,0,0,0,1034,77,171,-0.610722,0.881027,NONE


In [22]:
dataframe = df_data_1

In [12]:
defective = {'NONE':1, 'GAGEDEFECT':2, 'ALIGNDEFECT':3, 'GAGEALIGN':4, 'SURFACEDEFECT':5,'SURFACEALIGN':6}

In [25]:
dataframe[["defective"]] = pd.DataFrame([defective[item] for item in dataframe.DEFECTS])

In [26]:
dataframe.head()

,GAUGE,UNLD_GAUGE,SUPER_ELE,TWIST_A,TWIST_B,SURF79_L,SURF79_R,ALIGN79_L,ALIGN79_R,ALIGN_31_L,...,SURF_62_L,SURF_62_R,CURVE,S_IDX,CANT_L,CANT_R,WEB_L,WEB_R,DEFECTS,defective
0,976,303,0,0,0,0,0,0,0,0,...,0,0,0,963,75,179,-0.615378,0.967985,NONE,1
1,1021,305,0,0,0,0,0,0,0,0,...,0,0,0,981,55,163,-0.846215,0.783823,NONE,1
2,1140,325,0,0,0,0,0,0,0,0,...,0,0,0,999,80,159,-0.580411,0.752885,NONE,1
3,1120,335,0,0,0,0,0,0,0,0,...,0,0,0,1017,15,-40,0.000000,0.000000,NONE,1
4,1119,354,0,0,0,0,0,0,0,0,...,0,0,0,1034,77,171,-0.610722,0.881027,NONE,1


In [30]:
dataset = dataframe[["GAUGE","UNLD_GAUGE", "SUPER_ELE", "TWIST_A", "TWIST_B", "SURF79_L","SURF79_R", "ALIGN79_L","ALIGN79_R","ALIGN_31_L","ALIGN_31_R",
            "ALIGN_62_R", "SURF_31_L", "SURF_31_R", "SURF_62_L", "SURF_62_R" ,"CURVE", "CANT_L", "CANT_R", "WEB_L", "WEB_R", "DEFECTS" ]].values

In [33]:
X = dataset[:, 0:21].astype(float)
y = dataset[:,21]

In [37]:
seed =10
numpy.random.seed(seed)

In [39]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
dummy_y = np_utils.to_categorical(encoded_Y)  #dummy one-hot encoded y

In [53]:
import tensorflow as tf
from keras import backend as K

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [55]:
model = Sequential()
model.add(Dense(4, input_dim = 21, kernel_initializer = "normal", activation = "relu"))
model.add(Dense(6, kernel_initializer = "normal", activation = "softmax"))

# Build Model...

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[auc])

In [57]:
model.fit(X, dummy_y, epochs = 10, batch_size = 500)
model.save("my_model.h5")

Epoch 1/10
253602/253602 [==============================] - 5s 21us/step - loss: 0.0701 - auc: 0.9959
Epoch 2/10
253602/253602 [==============================] - 5s 21us/step - loss: 0.0692 - auc: 0.9963
Epoch 3/10
253602/253602 [==============================] - 5s 20us/step - loss: 0.0675 - auc: 0.9965
Epoch 4/10
253602/253602 [==============================] - 5s 21us/step - loss: 0.0659 - auc: 0.9967
Epoch 5/10
253602/253602 [==============================] - 5s 21us/step - loss: 0.0653 - auc: 0.9969
Epoch 6/10
253602/253602 [==============================] - 5s 21us/step - loss: 0.0640 - auc: 0.9971
Epoch 7/10
253602/253602 [==============================] - 5s 21us/step - loss: 0.0632 - auc: 0.9972
Epoch 8/10
253602/253602 [==============================] - 5s 21us/step - loss: 0.0622 - auc: 0.9973
Epoch 9/10
253602/253602 [==============================] - 5s 20us/step - loss: 0.0617 - auc: 0.9974
Epoch 10/10
253602/253602 [==============================] - 5s 20us/step - loss: 